# Create MoBIE Project

Create an example MoBIE project with the python mobie package.
See [the installation instructions](https://github.com/mobie/mobie-utils-python) to set up the python package.
For more details on the MoBIE and the MoBIE project structure check out [the MoBIE README](https://github.com/mobie/mobie#data-storage).

The data used in this example is part of the publication [Seipin and Nem1 establish discrete ER subdomains to initiate yeast lipid droplet biogenesis](https://doi.org/10.1083/jcb.201910177) and can be downloaded from [here](https://oc.embl.de/index.php/s/IV1709ZlcUB1k99).

In [6]:
# general imports
import os
import imageio
import mobie
from mobie.metadata import get_default_view

# the location of the data
# adapt these paths to your system and the input data you are using

# location of the input data. 
# the example data used in this notebook is available via this link:
# https://oc.embl.de/index.php/s/IV1709ZlcUB1k99
example_input_data = '/home/pape/Work/data/mobie/mobie-example-data'

# the location of the mobie project that will be created
# note that mobie project folders should always have the structure <PROECJT_ROOT_FOLDER/data>
# the folder 'data' will contain the sub-folders for individual datasets
mobie_project_folder = '/home/pape/Work/data/mobie/mobie_example_project/data'

# name of the dataset that will be created.
# one project can contain multiple datasets
dataset_name = 'example-dataset'

# the platform and number of jobs used for computation.
# choose 'local' to run computations on your machine.
# for large data, it is also possible to run computation on a cluster;
# for this purpose 'slurm' (for slurm cluster) and 'lsf' (for lsf cluster) are currently supported
target = 'local'
max_jobs = 4

## Initialize the dataset

First, we need to initialize the dataset. This step includes generating the top-level project folder (if it's not present already), the subfolders for the new dataset and adding the "default" image for this dataset.
All these steps are performed by the function `add_image`.

This function accepts input image data in different formats. The input data is specified with the arguments
`input_path`, which specifies the file path and `input_key`, which specifies the internal path or search patterns.
- tif images (2d or 3d) - for this option set `input_key=''`
- folder with image files - for this option `input_key` needs to be the glob pattern for the image files, e.g `input_key='*.tif'` to load all tif files
- hdf5 file - `input_key` needs to be the internal file path
- n5 or zarr file - `input_key` needs to be the internal file path

The input files will be copied into the project folder in the [bdv.n5 dataformat](https://github.com/bigdataviewer/bigdataviewer-core/blob/master/BDV%20N5%20format.md) and an image pyramid will be created through consecutive downsampling.

To efficiently process large files the inputs should be in hdf5, n5 or zarr format.
Note that all inputs need to be either 2d or 3d images (volumes).
Multi-channel images (volumes) should be seperated into their channels and then each channel added individually (see `Adding image data` below).

In [5]:
# The 'default' image for our example dataset is a 2d EM slice showing an overview of the dataset.
input_file = os.path.join(example_input_data, 'em_overview.tif')

# This is the name that will be given to the image source in mobie.
raw_name = 'em-raw'

# We need some metadata to create the n5-file in big-data-viewer format:
# - unit: the phyiscal unit of the coordinate system
# - resolution: the size of one voxel in the physical unit, this needs to be a tuple/list of length 3,
#               specifying the size for each of the 3 spatial dimensions
# - chunks: the size of the chunks (in voxels) that are used to store the output file.
#           good choices are usually (1, 512, 512) for 2d data and (64, 64, 64) for 3d data
# - scale_factors: the scale factors used for downsampling the input when creating the image pyramid
#                  this needs to be a list, where each entry specifies the scale factors for the 3 axes.
# Note that axes are always listed in the order ZYX here (in the java implementation of mobie / big-data-viewer the axis convention is XYZ).
# Also note that the values for all three axes (ZYX) need to be specified. In the case of 2d data, the value
# for Z should be set to 1.
unit = 'nanometer'
resolution = (1., 10., 10.)
chunks = (1, 512, 512)
scale_factors = 4 * [[1, 2, 2]]

mobie.add_image(
    input_path=input_file, 
    input_key='',  # the input is a single tif image, so we leave input_key blank
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=raw_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    is_default_dataset=True,  # mark this dataset as the default dataset that will be loaded by mobie
    target=target,
    max_jobs=max_jobs,
    unit=unit
)

DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example-dataset_em-raw, max_jobs=4, config_dir=tmp_example-dataset_em-raw/configs, target=local, dependency=DummyTask, input_path=tmp_example-dataset_em-raw/tmp_em_overview.h5, input_key=data, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [1.0, 10.0, 10.0], "unit": "nanometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-raw.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/

DEBUG: Pending tasks: 5
INFO: [pid 8387] Worker Worker(salt=935693922, workers=1, host=mokshi, username=pape, pid=8387) running   DownscalingLocal(tmp_folder=tmp_example-dataset_em-raw, max_jobs=4, config_dir=tmp_example-dataset_em-raw/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-raw.n5, input_key=setup0/timepoint0/s1, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-raw.n5, output_key=setup0/timepoint0/s2, scale_factor=(1, 2, 2), scale_prefix=s2, halo=[1, 2, 2], effective_scale_factor=[1, 4, 4], dependency=DownscalingLocal)
INFO: [pid 8387] Worker Worker(salt=935693922, workers=1, host=mokshi, username=pape, pid=8387) done      DownscalingLocal(tmp_folder=tmp_example-dataset_em-raw, max_jobs=4, config_dir=tmp_example-dataset_em-raw/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-raw.n5, input_key=setup0/timepoint0/s1, o

## Adding image data

After a dataset is created, we can add additional images to the dataset with the `add_image` function.

In [2]:
# First, we add two EM tomograms that are available in the example dataset.
# These tomograms show small areas in higher detail and in 3d.

# These are the two file names for the tomograms.
tomo_names = ['27_tomogram.tif', '29_tomogram.tif']

# We choose chunks and scale factors for 3d data, taking
# into account that the tomograms have a larger extent in the
# XY plane than in Z
unit = 'nanometer'
resolution = [5., 5., 5.]
chunks = (32, 128, 128)
scale_factors = [[1, 2, 2], [1, 2, 2],
                 [1, 2, 2], [1, 2, 2],
                 [2, 2, 2]]

# The tomograms need to be placed at the correct position w.r.t.
# the 2d em overview. This is achieved via an affine transformation,
# that has been determined externally and will be applied on the fly by big-data-viewer.
# Each affine transformation contains 12 parameters.
transformations = [
    [5.098000335693359, 0.0, 0.0, 54413.567834472655,
     0.0, 5.098000335693359, 0.0, 51514.319843292236,
     0.0, 0.0, 5.098000335693359, 0.0],
    [5.098000335693359, 0.0, 0.0, 39024.47988128662,
     0.0, 5.098000335693359, 0.0, 44361.50386505127,
     0.0, 0.0, 5.098000335693359, 0.0]
]

# add the two tomograms
for name, trafo in zip(tomo_names, transformations):
    im_name = f"em-{os.path.splitext(name)[0]}"
    im_path = os.path.join(example_input_data, name)
    
    # we need to pass additional 'view' arguments for the tomograms.
    # view arguments can modify the viewer state for loading the image source
    # here, we adjust the contrast limits to load the tomograms with
    # the correct contrast already and we set the affine trasnformtaiton
    # that will map the tomograms to the correct position via sourceTransforms
    im = imageio.volread(im_path)
    min_val, max_val = im.min(), im.max()
    source_transforms = [{"parameters": trafo}]
    view = get_default_view("image", im_name,
                            contrastLimits=[min_val, max_val],
                            sourceTransforms=source_transforms)
    
    mobie.add_image(
        input_path=im_path,
        input_key="",
        root=mobie_project_folder,
        dataset_name=dataset_name,
        image_name=im_name,
        resolution=resolution,
        scale_factors=scale_factors,
        transformation=trafo,
        chunks=chunks,
        target=target,
        max_jobs=max_jobs,
        view=view,
        unit=unit
    )

DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example-dataset_em-27_tomogram, max_jobs=4, config_dir=tmp_example-dataset_em-27_tomogram/configs, target=local, dependency=DummyTask, input_path=/home/pape/Work/data/mobie/mobie-example-data/27_tomogram.tif, input_key=, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2], [2, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2], [2, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [5.0, 5.0, 5.0], "unit": "nanometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-27_tomogram.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/pape

INFO: [pid 9162] Worker Worker(salt=685501593, workers=1, host=mokshi, username=pape, pid=9162) done      DownscalingLocal(tmp_folder=tmp_example-dataset_em-27_tomogram, max_jobs=4, config_dir=tmp_example-dataset_em-27_tomogram/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-27_tomogram.n5, input_key=setup0/timepoint0/s0, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-27_tomogram.n5, output_key=setup0/timepoint0/s1, scale_factor=(1, 2, 2), scale_prefix=s1, halo=[1, 2, 2], effective_scale_factor=[1, 2, 2], dependency=CopyVolumeLocal)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   DownscalingLocal_tmp_example_data_CopyVolumeLocal__1__2__2__d99f555215   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 6
INFO: [pid 9162] Worker Worker(salt=685501593, workers=1, host=mokshi, username=pape, pid=9162) running   Do

INFO: Informed scheduler that task   WriteDownscalingMetadata_DownscalingLocal___resolution_____bdv_n5_c398304e7c   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 9162] Worker Worker(salt=685501593, workers=1, host=mokshi, username=pape, pid=9162) running   DownscalingWorkflow(tmp_folder=tmp_example-dataset_em-27_tomogram, max_jobs=4, config_dir=tmp_example-dataset_em-27_tomogram/configs, target=local, dependency=DummyTask, input_path=/home/pape/Work/data/mobie/mobie-example-data/27_tomogram.tif, input_key=, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2], [2, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2], [2, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [5.0, 5.0, 5.0], "unit": "nanometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-27_tomogram.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0)
INFO: [pid 9162] Worke

DEBUG: Checking if CopyVolumeLocal(tmp_folder=tmp_example-dataset_em-29_tomogram, max_jobs=4, config_dir=tmp_example-dataset_em-29_tomogram/configs, input_path=/home/pape/Work/data/mobie/mobie-example-data/29_tomogram.tif, input_key=, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-29_tomogram.n5, output_key=setup0/timepoint0/s0, prefix=initial_scale, dtype=None, fit_to_roi=False, effective_scale_factor=[], dependency=DummyTask) is complete
INFO: Informed scheduler that task   DownscalingLocal_tmp_example_data_CopyVolumeLocal__1__2__2__2f2e308569   has status   PENDING
DEBUG: Checking if DummyTask() is complete
INFO: Informed scheduler that task   CopyVolumeLocal_tmp_example_data_DummyTask_None_aeabe50ec6   has status   PENDING
INFO: Informed scheduler that task   DummyTask__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 8
INFO: [

DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   DownscalingLocal_tmp_example_data_DownscalingLocal__1__16__16__7ca0e81d46   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 9162] Worker Worker(salt=392726061, workers=1, host=mokshi, username=pape, pid=9162) running   DownscalingLocal(tmp_folder=tmp_example-dataset_em-29_tomogram, max_jobs=4, config_dir=tmp_example-dataset_em-29_tomogram/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-29_tomogram.n5, input_key=setup0/timepoint0/s4, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-29_tomogram.n5, output_key=setup0/timepoint0/s5, scale_factor=(2, 2, 2), scale_prefix=s5, halo=[2, 2, 2], effective_scale_factor=[2, 32, 32], dependency=DownscalingLocal)
INFO: [pid 9162] Worker Worker(salt=392726061, workers=1, host=mokshi, username=pape, pid=9162) done  

In [10]:
# Next, we add a fluorescence image that is also part of the example dataset.

input_path = os.path.join(example_input_data, 'fluorescence_downsampled.tif')

# The name of the image in mobie.
# Note that mobie will use the identifier in front of the first '-'
# to group images by name.
# So in this case we will have the two groups 'em' and 'lm'.
im_name = "lm-fluorescence"

# This is again a 2d image, so we set all values for Z to 1.
unit = 'nanometer'
resolution = [1., 100., 100.]
scale_factors = [[1, 2, 2], [1, 2, 2], [1, 2, 2]]
chunks = (1, 512, 512)

# we set the default display color to green.
view = get_default_view(
    "image", im_name,
    color="green"
)

mobie.add_image(
    input_path=input_path,
    input_key="",
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=im_name,
    resolution=resolution,
    scale_factors=scale_factors,
    view=view,
    chunks=chunks,
    target=target,
    max_jobs=max_jobs,
    unit=unit
)

DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example-dataset_lm-fluorescence, max_jobs=4, config_dir=tmp_example-dataset_lm-fluorescence/configs, target=local, dependency=DummyTask, input_path=tmp_example-dataset_lm-fluorescence/tmp_fluorescence_downsampled.h5, input_key=data, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [1.0, 100.0, 100.0], "unit": "nanometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/lm-fluorescence.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/pape/Work/software/conda/minico

INFO: [pid 9162] Worker Worker(salt=103174090, workers=1, host=mokshi, username=pape, pid=9162) done      DownscalingLocal(tmp_folder=tmp_example-dataset_lm-fluorescence, max_jobs=4, config_dir=tmp_example-dataset_lm-fluorescence/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/lm-fluorescence.n5, input_key=setup0/timepoint0/s1, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/lm-fluorescence.n5, output_key=setup0/timepoint0/s2, scale_factor=(1, 2, 2), scale_prefix=s2, halo=[1, 2, 2], effective_scale_factor=[1, 4, 4], dependency=DownscalingLocal)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   DownscalingLocal_tmp_example_data_DownscalingLocal__1__4__4__e83b98ce1a   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 9162] Worker Worker(salt=103174090, workers=1, host=mokshi, username=pape, pid=9162) runnin

In [12]:
# as last image, we add a binary mask for the foreground in the image
input_path = os.path.join(example_input_data, 'em_mask.tif')
mask_name = "em-mask"

# again, the mask is 2d
unit = "nanometer"
chunks = [1, 256, 256]
resolution = [1., 160., 160.]
scale_factors = [[1, 2, 2]]

mobie.add_image(
    input_path=input_path,
    input_key="",
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=mask_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    unit=unit
)

DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example-dataset_em-mask, max_jobs=4, config_dir=tmp_example-dataset_em-mask/configs, target=local, dependency=DummyTask, input_path=tmp_example-dataset_em-mask/tmp_em_mask.h5, input_key=data, scale_factors=[[1, 2, 2]], halos=[[1, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [1.0, 160.0, 160.0], "unit": "nanometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-mask.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "sc

DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   DownscalingWorkflow_tmp_example_data_DummyTask_False_8a45295332   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=378584402, workers=1, host=mokshi, username=pape, pid=9162) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 5 tasks of which:
* 1 complete ones were encountered:
    - 1 DummyTask()
* 4 ran successfully:
    - 1 CopyVolumeLocal(...)
    - 1 DownscalingLocal(...)
    - 1 DownscalingWorkflow(...)
    - 1 WriteDownscalingMetadata(...)

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



## Adding segmentation data

In addition to image data and masks, MoBIE supports segmentations, which contain label masks for different objects
(e.g. organs, cells, ultrastructure) in the volume. For segmentations, MoBIE also supports tables, which contain additional properties for the objects in the segmentation.
The function `add_segmentation` copies the input data and also generates the default table for the segmentation.

In [13]:
# we add a segmentation for several objects visible in the em-overview image
input_path = os.path.join(example_input_data, 'em_segmentation.tif')
segmentation_name = "em-segmentation"

unit = "nanometer"
resolution = [1., 30., 30.]
chunks = [1, 256, 256]
scale_factors = [[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]]

mobie.add_segmentation(
    input_path=input_path,
    input_key="",
    root=mobie_project_folder,
    dataset_name=dataset_name,
    segmentation_name=segmentation_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    add_default_table=True  # add the default table with the properties mobie needs to interact with table and segmentation
)

DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, target=local, dependency=DummyTask, input_path=tmp_example-dataset_em-segmentation/tmp_em_segmentation.h5, input_key=data, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], metadata_format=bdv.n5, metadata_dict={"resolution": [1.0, 30.0, 30.0], "unit": "micrometer"}, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-segmentation.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/home/pape/Work/software/conda/miniconda3/envs/main/lib/python3.7/site-packages/luigi/parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/home/pape/Work/software/

DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   DownscalingLocal_tmp_example_data_CopyVolumeLocal__1__2__2__236d787f38   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 5
INFO: [pid 9162] Worker Worker(salt=087543433, workers=1, host=mokshi, username=pape, pid=9162) running   DownscalingLocal(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, input_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-segmentation.n5, input_key=setup0/timepoint0/s1, output_path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-segmentation.n5, output_key=setup0/timepoint0/s2, scale_factor=(1, 2, 2), scale_prefix=s2, halo=[1, 2, 2], effective_scale_factor=[1, 4, 4], dependency=DownscalingLocal)
INFO: [pid 9162] Worker Worker(salt=087543433, workers=1, host=mokshi, username=pape, pid=9162) done   

DEBUG: Checking if DataStatisticsWorkflow(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, target=local, dependency=DummyTask, path=/home/pape/Work/data/mobie/mobie_example_project/data/example-dataset/images/local/em-segmentation.n5, key=setup0/timepoint0/s0, output_path=tmp_example-dataset_em-segmentation/statistics.json) is complete
DEBUG: Checking if MergeStatisticsLocal(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, output_path=tmp_example-dataset_em-segmentation/statistics.json, shape=[1, 3533, 3444], dependency=BlockStatisticsLocal) is complete
INFO: Informed scheduler that task   DataStatisticsWorkflow_tmp_example_data_DummyTask_setup0_timepoint_5a51f35f03   has status   PENDING
DEBUG: Checking if BlockStatisticsLocal(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, path=/home/p

DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   BlockMorphologyLocal_tmp_example_data_DummyTask_setup0_timepoint_f3d69128c0   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 9162] Worker Worker(salt=712312784, workers=1, host=mokshi, username=pape, pid=9162) running   MergeMorphologyLocal(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, input_path=tmp_example-dataset_em-segmentation/data.n5, input_key=morphology_attributes, output_path=tmp_example-dataset_em-segmentation/data.n5, output_key=attributes, number_of_labels=460, prefix=attributes, dependency=BlockMorphologyLocal)
INFO: [pid 9162] Worker Worker(salt=712312784, workers=1, host=mokshi, username=pape, pid=9162) done      MergeMorphologyLocal(tmp_folder=tmp_example-dataset_em-segmentation, max_jobs=4, config_dir=tmp_example-dataset_em-segmentation/configs, input_path=tmp_example

## Publishing the project 

The project created above will be located on the local filesystem at `mobie_project_folder`.
In order to share it with collaborators or make the data public, MoBIE can also read data stored in a
[AWS S3](https://aws.amazon.com/s3/) compatible object store.
For this, some additional metadata is necessary, that can be generated via `add_remote_project_metadata`.

The data then needs to be uploaded to the s3 storage by some appropriate tool and the metadata needs to be uploaded to github to make it accessible for MoBIE.

In [14]:
from mobie.metadata import add_remote_project_metadata

# to generate the metadata for publishing the project, the
# following information is needed:
# - bucket_name: the name of the bucket in the object store
# - service_endpoint: the address of the service endpoint used.
#                     this allows specifying object stores that are different from aws
#                     here, we use the object store located at EMBL Heidelberg as service endpoint.
#                     to use an aws s3 endpoint, set it to https://s3.amazonaws.com 
bucket_name = 'my-test-bucket'

service_endpoint = 'https://s3.embl.de'

add_remote_project_metadata(
    mobie_project_folder,
    bucket_name,
    service_endpoint
)

# Once the metadata is generated, you can upload your project. 
# MoBIE can access projects directly from an s3 compatible object store.
# Optionally the metadata can be uploaded to github to have it under version control;
# the github repository can also be used as entry point for the MoBIE viewer.

# 1.) Upload the complete folder at "mobie_project_folder" to the s3 bucket.
# There are several tools available to achieve this, for example
# aws s3 sync (https://docs.aws.amazon.com/cli/latest/reference/s3/sync.html)
# The sync command would look something like this (assuming the file paths used in this example)
# $ aws s3 sync /home/pape/Work/data/mobie/mobie_example_project/data https://s3.embl.de/my-test-bucket

# 2.) (OPTIONAL!) Create a github repository for this project and upload the metadata to it:
# - Go to https://github.com/ and log into or create your account
# - Create a new empty (!) repository, e.g. called "my-mobie-project"
# - Go to /home/pape/Work/data/mobie_example_project in a terminal (again assuming the filepaths used in the example notebook)
# - Initialize git via 
#   $ git init
# - Add the repository you just created as remote via
#   $ git remote add origin https://github.com/<USERNAME>/my-mobie-project
# - Tell git to ignore the image data files (n5 files) by creating a file ".gitignore" and adding the line "*.n5"
#   This is very important, because otherwise we would add all the image data to git.
# - Add the metadata to git via
#   $ git add .
# - Upload the data to github via
#   $ git push origin master

## Troubleshooting

TODO